In [53]:
import reactionmodel.load
import glob
import os
import re
import numpy as np
import pandas as pd

from simulators import SIMULATORS

inital_cwd = os.getcwd()

In [ ]:
os.chdir(inital_cwd)
test_dir = "./tests/sbml-tests/"
tests = glob.glob(os.path.join(test_dir, 'sbml-*'))

target_check = "p01ic01"
target_catchment = []

specifications = {}

for test in tests:
    run_tests_from_dir(test)

In [132]:
def do_simulations(s, n=1000):
    results = []
    print(s)
    simulator = s.options['simulator']
    if simulator == 'hybrid':
        from hybrid import partition_by_threshold
        simulator = SIMULATORS[simulator]
        t_span = [0.0, 50.0]
        discontinuities = np.linspace(0, 50, 51)
        rng = np.random.default_rng()
        partition_function = lambda p: partition_by_threshold(p, 100)
        s.model.bake_k(parameters=s.parameters, jit=True)
        simulation_options = s.options.copy()
        simulation_options.pop('simulator')
        for i in range(n):
            print(i)
            result = simulator(s.initial_condition, t_span, s.model.k_jit, s.model.stoichiometry(), s.model.rate_involvement(), rng, discontinuities=discontinuities, partition_function=partition_function, **simulation_options)
            results.append(result)
    return results

def run_tests_with_checks(root, specifications):
    tests_to_do = []
    for check in glob.glob(os.path.join(root, 'checks', '*/')):
        check_dir_root = check.split('/')[-2]
        if target_check is not None:
            if check_dir_root != target_check:
                print(f"Since we are targeting a single check, we are skipping {check_dir_root}")
                continue
        tests_to_do.append((specifications[check_dir_root], check))
    print(f"Performing tests with benchmark data for {len(tests_to_do)}/{len(specifications)} combinations.")

    for specification,check_dir in tests_to_do:
        check_data = pd.read_csv(glob.glob(os.path.join(check_dir, '*.csv'))[0])
        #print(check_data)
        results = do_simulations(specification)
        desired_species = set([c.split('-')[0] for c in check_data.columns if len(c.split('-')) > 1])
        assert len(desired_species) == 1
        desired_species = list(desired_species)[0]
        target_index = [s.name for s in specification.model.species].index(desired_species)
        print("TARGET INDEX: ", target_index)
        aligned = align_results(results, check_data['time'], target_index, desired_species)
        if target_check:
            print("Catching our targeted check in target_catchment")
            target_catchment.append((aligned, check_data))
        # to csv
        return aligned

def align_results(results, time, target_index, species_name):
    all_aligned = []
    for r in results:
        aligned = []
        t_history = r.t_history
        for t in time[:-1]:
            idx = np.argmin(np.abs(t-t_history))
            aligned.append((r.t_history[idx], r.y_history[target_index,idx]))
        aligned.append((r.t, r.y[target_index]))
        all_aligned.append(pd.DataFrame.from_records(aligned, columns=['time', species_name]))
    return all_aligned

def run_tests_from_dir(dir):
    model_path = os.path.join(dir, 'model.txt')
    for params_path in glob.glob(os.path.join(dir, 'parameters', 'parameters*.txt')):
        for config_path in glob.glob(os.path.join(dir, 'configurations', 'config*.txt')):
            for ic_path in glob.glob(os.path.join(dir, 'initial_conditions', 'initial*.txt')):
                specification = reactionmodel.load.load_specification(model_path, params_path, config_path, ic_path)
                # use the parameter and ic file names as a unique identifier for this combination
                # later, we will look up all the combinations that we have test data for, and run simulations to check
                param_id = re.search('[a-z]+([0-9]+)\.txt', params_path)[1]
                ic_id = re.search('[a-z]+([0-9]+)\.txt', ic_path)[1]
                specifications[f'p{param_id}ic{ic_id}'] = specification
    
    run_tests_with_checks(dir, specifications)

# Analyze results

In [134]:
single_target = target_catchment[0]
results, check = single_target

In [135]:
indexed_results = []
for r in results:
    r['time'] = np.round(r['time'], 5)
    r = r.set_index('time')
    indexed_results.append(r)

In [136]:
results_to_check = pd.concat([pd.concat(indexed_results, axis=1).mean(axis=1), pd.concat(indexed_results, axis=1).std(axis=1)], axis=1)
results_to_check.columns=['mean', 'std']
results_to_check

,mean,std
time,,
0.0,100.000,0.000000
1.0,99.316,4.565851
2.0,98.515,6.468045
3.0,97.509,7.728107
4.0,96.328,9.011749
5.0,95.399,10.036161
6.0,94.487,10.804562
7.0,93.605,11.631348
8.0,92.705,12.337917


In [137]:
# https://github.com/sbmlteam/sbml-test-suite/blob/release/cases/stochastic/DSMTS-userguide-31v2.pdf
z_t = (results_to_check['mean'] - check['X-mean'])/(check['X-sd']) * np.sqrt(1000)

In [138]:
z_t

time
0.0          NaN
1.0     2.162397
2.0     2.452479
3.0     1.892483
4.0     0.885480
5.0     0.884448
6.0     0.915076
7.0     1.004911
8.0     1.018935
9.0     1.697400
10.0    2.105335
11.0    2.201587
12.0    2.093926
13.0    2.158640
14.0    2.314183
15.0    2.053151
16.0    2.007496
17.0    1.794685
18.0    1.828670
19.0    1.479595
20.0    1.337925
21.0    1.337694
22.0    1.501336
23.0    1.389620
24.0    0.999184
25.0    1.072238
26.0    0.968755
27.0    0.794049
28.0    0.863551
29.0    0.989982
30.0    1.034944
31.0    0.929074
32.0    0.893041
33.0    0.739350
34.0    0.904931
35.0    0.779325
36.0    0.568518
37.0    0.351713
38.0    0.409408
39.0    0.515497
40.0    0.435973
41.0    0.538910
42.0    0.548510
43.0    0.641265
44.0    0.683660
45.0    0.753941
46.0    0.867687
47.0    1.078696
48.0    0.986978
49.0    0.938256
50.0    0.838965
dtype: float64